In [ ]:
import librosa
import numpy as np
from numpy.fft import rfft
from numpy import pi
from matplotlib import pyplot as plt
from IPython.display import Audio
import cmath
import scipy
from cache_no_hash import cache
from blindDescend import blindDescend
from yin import yin
from harmonicSynth import HarmonicSynth, Harmonic

TWO_PI = np.pi * 2

In [ ]:
PAGE_LEN = 1024
SR = 22050
DTYPE = np.float32
N_HARMONICS = 8

In [ ]:
HANN = scipy.signal.get_window('hann', PAGE_LEN, True)
IMAGINARY_LADDER = np.linspace(0, TWO_PI * 1j, PAGE_LEN)
SPECTRUM_SIZE = PAGE_LEN // 2 + 1
NYQUIST = SR // 2

In [ ]:
def sino(freq, length):
    return np.sin(np.arange(length) * freq * TWO_PI / SR)

def playHard(data):
    return Audio(data, rate = SR)
def play(data, soft = .1):
    t = np.concatenate([data, [1]])
    length = round(soft * SR)
    t[:length ] = np.multiply(t[:length ], np.linspace(0, 1, length))
    t[-length:] = np.multiply(t[-length:], np.linspace(1, 0, length))
    return playHard(t)

def findPeaks(energy):
    slope = np.sign(energy[1:] - energy[:-1])
    extrema = slope[1:] - slope[:-1]
    return np.argpartition(
        (extrema == -2) * energy[1:-1], - N_HARMONICS,
    )[- N_HARMONICS:] + 1

def sft(signal, freq_bin):
    # Slow Fourier Transform
    return np.abs(np.sum(signal * np.exp(IMAGINARY_LADDER * freq_bin))) / PAGE_LEN

def refineGuess(guess, signal):
    def loss(x):
        if x < 0:
            return 0
        return - sft(signal, x)
    freq_bin, loss = blindDescend(loss, .01, .4, guess)
    return freq_bin * SR / PAGE_LEN, - loss

def widePlot(h = 3, w = 12):
    plt.gcf().set_size_inches(w, h)

    
def spectro(signal, do_wide = True, trim = 130):
    energy = np.abs(rfft(signal * HANN))
    plt.plot(energy[:trim])
    if do_wide:
        widePlot()

def concatSynth(synth, harmonics, n):
    buffer = []
    for i in range(n):
        synth.eat(harmonics)
        buffer.append(synth.mix())
    return np.concatenate(buffer)

def pitch2freq(pitch):
    return np.exp((pitch + 36.37631656229591) * 0.0577622650466621)

def freq2pitch(f):
    return np.log(f) * 17.312340490667562 - 36.37631656229591

def pagesOf(signal):
    for i in range(0, signal.size - PAGE_LEN + 1, PAGE_LEN):
        yield signal[i : i + PAGE_LEN]


In [ ]:
y, sr = librosa.load('freesound.wav')
assert sr == SR

In [ ]:
play(y)

In [ ]:
def plotF0():
    f = []
    for p in pagesOf(y):
        f0 = yin(p, sr, PAGE_LEN)
        f.append(f0)
    plt.plot(f)
plotF0()
print(yin(y, sr, len(y)))

In [ ]:
SR / 372.4, SR / 372

In [ ]:
N_FRAMES_PERIOD = 59
plt.plot(y[len(y)//2:][:N_FRAMES_PERIOD])

In [ ]:
def temporalIntegrate(signal):
    return np.mean(np.abs(signal))
print(temporalIntegrate(sino(SR // 4, SR)))
print(temporalIntegrate(sino(SR // 8, SR)))
print(temporalIntegrate(sino(SR // 4, SR) + sino(SR // 8, SR)))
print(temporalIntegrate(sino(SR // 4, SR) + sino(SR // 4, SR)))

It is only meaningful for same-timbre sounds. 

In [ ]:
def plotAmp():
    A = []
    for i in range(0, y.size - N_FRAMES_PERIOD + 1, N_FRAMES_PERIOD):
        signal = y[i : i + N_FRAMES_PERIOD]
        a = temporalIntegrate(signal)
        A.append(a)
    fig, (ax0, ax1) = plt.subplots(2, 1)
    ax0.plot(A)
    ax1.plot(A[:50])
plotAmp()
widePlot(5)

In [ ]:
attack = 8 * N_FRAMES_PERIOD / SR
attack

In [ ]:
spectrum = np.abs(rfft(y))
freq_bins = np.linspace(0, SR / 2, len(spectrum))
def plotSpec(start_f = 0, end_f = SR // 2, trans = np.log, **kw):
    start = round(start_f / (SR / 2) * len(spectrum))
    end   = round(end_f   / (SR / 2) * len(spectrum))
    plt.plot(freq_bins[start:end], trans(spectrum[start:end]), **kw)
plotSpec()
f0 = 373
walls = [
    *range(round(f0 * .5), f0 * 9, f0), 
    *range(round(f0 * 9.7), f0 * 15, round(f0 * 1.1)), 
    *range(round(f0 * 15.3), f0 * 28, round(f0 * 1.2)), 
]
for f in walls:
    plt.axvline(f, c='r')
widePlot()

In [ ]:
N_HARMONICS = len(walls) - 1
N_HARMONICS

In [ ]:
plotSpec(369, 372, trans=lambda x:x)
5

In [ ]:
plotSpec(741.4, 743.5, trans=lambda x:x)
3

In [ ]:
plotSpec(4205, 4214, trans=lambda x:x)

In [ ]:
def findPeaks(diameter = 5):
    radius = (diameter - 1) // 2
    freqs    = []
    energies = []
    for left, right in zip(walls[:-1], walls[1:]):
        max_e = 0
        max_i = None
        left_i  = round(left  / (SR / 2) * len(spectrum))
        right_i = round(right / (SR / 2) * len(spectrum))
        for i, energy in enumerate(spectrum[left_i:right_i]):
            if energy > max_e:
                max_e = energy
                max_i = i
        freqs.append(
            freq_bins[left_i + max_i]
        )
        energies.append(
            np.sqrt(np.sum(spectrum[left_i:][
                max_i - radius : max_i + radius + 1
            ] ** 2))
        )
    return freqs, energies
def benchmark():
    freqs, energies = findPeaks()
    plotSpec()
    plt.plot(freqs, np.log(energies), c='r')
benchmark()

In [ ]:
from scipy.interpolate import interp1d
def getEnvelope():
    freqs, energies = findPeaks()
    f = interp1d([0, *freqs], [energies[0] ,*energies])
    def envelope(x):
        try:
            return f(x)
        except ValueError:
            return 0
    return envelope
envelope = getEnvelope()
def benchmarkEnv(trans = np.log):
    plotSpec(trans = trans)
    plt.plot(freq_bins, trans([envelope(x) for x in freq_bins]))
# benchmarkEnv(lambda x:x)
benchmarkEnv()

In [ ]:
from harmonicSynth import HarmonicSynth, Harmonic

In [ ]:
assert attack * SR < PAGE_LEN
attack * SR, PAGE_LEN

In [ ]:
def natureHarmonics(f0):
    return np.arange(N_HARMONICS) * f0
def synth(f, t = 2, decayRate = 1, harmo_way = natureHarmonics):
    n_pages = round(t * SR / PAGE_LEN)
    amp_env = np.exp(np.linspace(0, - t * decayRate, n_pages * PAGE_LEN))
    attack_frames = round(attack * SR)
    amp_env[:attack_frames] = np.linspace(
        0, amp_env[attack_frames - 1], attack_frames, 
    )
    hS = HarmonicSynth(N_HARMONICS, SR, PAGE_LEN, np.float32, True, False)
    harmonics = [Harmonic(f, envelope(f)) for f in harmo_way(f0)]
    hS.eat(harmonics)
    hS.mix()
    buffer = []
    for page_i in range(n_pages):
        hS.eat(harmonics)
        buffer.append(hS.mix())
    signal = np.concatenate(buffer)
    return signal * amp_env

In [ ]:
playHard(synth(f0, t = len(y) / SR, decayRate=1.5))

In [ ]:
playHard(y)

Including the harmonic drift...

In [ ]:
def weirdHarmo(_):
    freqs, _ = findPeaks()
    return freqs
playHard(synth(f0, t = len(y) / SR, decayRate=1.5, harmo_way = weirdHarmo))

Attack and harmonics decay/swing are not good. 